# Предобработка текста

In [1]:
cd ../

/Users/fulyankin/Documents/massResearch_houses/humanity_code


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [3]:
import gensim 
print(gensim.__version__)

from gensim.models.word2vec import Word2Vec

3.8.0


In [6]:
from texts import parallel_prepare_text, prepare_text, tag_mystem

In [5]:
dff = pd.read_csv('cian_data_v1.csv', sep='\t')
T = dff["описание"].values

## Лемматизация 

In [ ]:
T_lemm = parallel_prepare_text(T, prepare_text)

 94%|█████████▍| 34/36 [04:37<00:14,  7.49s/it]

In [13]:
!touch description_lemm.txt

In [14]:
f = open('description_lemm.txt', "w")

for line in T_lemm:
    f.write(' '.join(line) + '\n')
    
f.close() 

In [15]:
!head -n 1 description_lemm.txt

очень уютный красивый квартира хороший ремонт встроить кухня весь инфраструктура шаговый доступность полный стоимость договор торг


In [18]:
f =  open('description_lemm.txt', "r")
x = f.readlines()
f.close()

len(x)

35109

## Тэгирование 

In [7]:
T_lemm = parallel_prepare_text(T, tag_mystem)

100%|██████████| 36/36 [01:19<00:00,  2.20s/it]


In [9]:
!touch description_tag.txt

In [14]:
f = open('description_tag.txt', "w")

for line in T_lemm:
    f.write(' '.join(line) + '\n')
    
f.close() 

In [16]:
!head -n 1 description_tag.txt

очень_ADV уютный_ADJ красивый_ADJ квартира_NOUN хороший_ADJ ремонт_NOUN встраивать_VERB кухня_NOUN весь_DET инфраструктура_NOUN в_ADP шаговый_ADJ доступность_NOUN полный_ADJ стоимость_NOUN в_ADP договор_NOUN торг_NOUN


In [17]:
f =  open('description_tag.txt', "r")
x = f.readlines()
f.close()
len(x)

35109

# Обучение w2v 

In [25]:
T_lemm = [x.split(" ") for x in T_lemm]

In [26]:
model = Word2Vec(size=300, window=2, min_count=3, workers=-1)
model.build_vocab(T_lemm)
model.train(T_lemm, total_examples=model.corpus_count, epochs=1000)

(0, 0)

In [36]:
#model.save('Ivanw2v.bin')

In [29]:
model.wv.most_similar('квартира')

[('инфоаструктура', 0.23434817790985107),
 ('есенин', 0.22186075150966644),
 ('увлажнить', 0.21443741023540497),
 ('кубанский', 0.20375016331672668),
 ('музейный', 0.19780336320400238),
 ('интерьерный', 0.1858503520488739),
 ('2.65м', 0.18252192437648773),
 ('медпункт', 0.18231607973575592),
 ('черёмушка', 0.1822584867477417),
 ('москва2020', 0.18039485812187195)]

In [33]:
cnt['инфоаструктура']

7

In [35]:
cnt['инфраструктура']

17162

In [34]:
from collections import Counter
cnt = Counter([item for sublist in T_lemm for item in sublist])
cnt.most_common()[:20]

[('квартира', 65691),
 ('дом', 42810),
 ('детский', 26645),
 ('комплекс', 23858),
 ('метро', 20032),
 ('комната', 19648),
 ('окно', 18379),
 ('этаж', 18366),
 ('доступность', 18027),
 ('двор', 18024),
 ('школа', 17791),
 ('сад', 17302),
 ('район', 17299),
 ('инфраструктура', 17162),
 ('минута', 16831),
 ('территория', 15631),
 ('продаваться', 15534),
 ('расположить', 15305),
 ('жк', 15293),
 ('площадь', 14681)]

# Распаковка w2v 

In [45]:
from tqdm import tqdm

In [41]:
# w2v_model = model

MODEL_PATH = "./Ivanw2v.bin"
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(MODEL_PATH, binary=True)

In [ ]:
w2v_model.wv.get_vector("путин_NOUN")[:5]

In [47]:
d = w2v_model.vector_size
V = len(w2v_model.wv.vocab) + 2

# "#UNK" для неизвестных слов
# "#PAD" заглушки, чтобы были одинаковые размерности у матриц

# Мама мыла раму 
# Филипп закомитил код на гитхаб  

# Мама мыла раму  #PAD  #PAD
# #UNK закомитил код на #UNK

word2index = {"#UNK": 0, "#PAD":1}

embedding = np.zeros((V, d))

for i,word in tqdm(enumerate(w2v_model.wv.vocab.keys())):
    word2index[word] = i + 2
    embedding[i+2] = w2v_model.wv.get_vector(word)
    
assert embedding.shape == (V, d)
assert len(word2index) == V

15203it [00:00, 158635.71it/s]


In [48]:
np.save("embedding.npy", embedding)

In [49]:
import pickle 

with open("word2index.pickle", "wb") as f:
    pickle.dump(word2index, f)